<a href="https://colab.research.google.com/github/mohd-faizy/CAREER-TRACK-Machine-Learning-Scientist-with-Python/blob/main/03_Cross_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

--- 
<strong> 
    <h1 align='center'>Cross Validation</h1> 
</strong>

---

Holdout sets are a great start to model validation. However, using a single train and test set if often not enough. Cross-validation is considered the gold standard when it comes to validating model performance and is almost always used when tuning model hyper-parameters. This chapter focuses on performing cross-validation to validate model performance.

In [1]:
!git clone https://github.com/mohd-faizy/CAREER-TRACK-Machine-Learning-Scientist-with-Python.git

Cloning into 'CAREER-TRACK-Machine-Learning-Scientist-with-Python'...
remote: Enumerating objects: 686, done.
remote: Counting objects: 100% (357/357), done.
remote: Compressing objects: 100% (317/317), done.
remote: Total 686 (delta 91), reused 284 (delta 38), pack-reused 329
Receiving objects: 100% (686/686), 202.35 MiB | 24.66 MiB/s, done.
Resolving deltas: 100% (217/217), done.
Checking out files: 100% (314/314), done.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

plt.style.use('fivethirtyeight')
#plt.style.use('ggplot')
#sns.set_theme()

%matplotlib inline

In [3]:
os.chdir('/content/CAREER-TRACK-Machine-Learning-Scientist-with-Python/10_Model_Validation_in_Python/_dataset')
cwd = os.getcwd()
print('Curent working directory is ', cwd)

Curent working directory is  /content/CAREER-TRACK-Machine-Learning-Scientist-with-Python/10_Model_Validation_in_Python/_dataset


In [4]:
ls

candy-data.csv  sports.csv  tic-tac-toe.csv


## The problems with holdout sets

### Two samples
After building several classification models based on the `tic_tac_toe` dataset, you realize that some models do not generalize as well as others. You have created training and testing splits just as you have been taught, so you are curious why your validation process is not working.

After trying a different training, test split, you noticed differing accuracies for your machine learning model. Before getting too frustrated with the varying results, you have decided to see what else could be going on.

In [5]:
tic_tac_toe = pd.read_csv('tic-tac-toe.csv')
tic_tac_toe.head()

,Top-Left,Top-Middle,Top-Right,Middle-Left,Middle-Middle,Middle-Right,Bottom-Left,Bottom-Middle,Bottom-Right,Class
0,x,x,x,x,o,o,x,o,o,positive
1,x,x,x,x,o,o,o,x,o,positive
2,x,x,x,x,o,o,o,o,x,positive
3,x,x,x,x,o,o,o,b,b,positive
4,x,x,x,x,o,o,b,o,b,positive


In [6]:
# Create two different samples of 200 observations
sample1 = tic_tac_toe.sample(n=200, random_state=1111)
sample2 = tic_tac_toe.sample(n=200, random_state=1171)

# Print the number of common observations
print(len([index for index in sample1.index if index in sample2.index]))

# Print the number of observations in the Class column for both samples
print(sample1['Class'].value_counts())
print(sample2['Class'].value_counts())

40
positive    134
negative     66
Name: Class, dtype: int64
positive    123
negative     77
Name: Class, dtype: int64


> __Notice that there are a varying number of positive observations for both sample test sets. Sometimes creating a single test holdout sample is not enough to achieve the high levels of model validation you want. You need to use something more robust.__

**Potential problems with holdout samples:**

- *Using different data splitting methods may lead to varying data in the final holdout samples.*

- *we learned that our accuracy metric on this validation set may be misleading, or if we split this data differently, we might get different results.*


## Cross-validation

**Cross-validation** is a **resampling** procedure used to **evaluate** machine learning models on a **limited data sample**. The procedure has a single parameter called __k__ that refers to the number of groups that a given data sample is to be __split__ into. As such, the procedure is often called __k-fold cross-validation__.

<p align='center'>
    <a href='#'><img src='https://scikit-learn.org/stable/_images/grid_search_cross_validation.png'></a>
</p>

### scikit-learn's `KFold()`

You just finished running a colleagues code that creates a **random forest model** and calculates an out-of-sample accuracy. You noticed that your colleague's code did not have a random state, and the errors you found were completely different than the errors your colleague reported.

To get a better estimate for how accurate this **random forest model** will be on new data, you have decided **to generate some indices to use for KFold cross-validation**.

In [7]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error 

X = np.array(range(40))
y = np.array([0]*20 + [1]*20)

kf = KFold(n_splits=5)
splits = kf.split(X)

for train_index, test_index in splits:
    print(len(train_index), len(test_index))

# Print one of the index sets:
print(train_index, test_index)

# Intensiating the model
rfc = RandomForestRegressor(n_estimators=25, random_state=1111)
for train_index, val_index in splits:
    # Setup the training and validation data
    X_train, y_train = X[train_index], y[train_index]
    X_val, y_val = X[val_index], y[val_index]
    
    # Fit the random forest model
    rfc.fit(X_train, y_train)
    
    # Make predictions, and print the accuracy
    y_pred = rfc.predict(X_test)
    print("Split accuracy: ",mean_squared_error(y_val, y_pred))

32 8
32 8
32 8
32 8
32 8
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31] [32 33 34 35 36 37 38 39]


In [8]:
candy = pd.read_csv('candy-data.csv')

X = candy.drop(['competitorname', 'winpercent'], axis=1).to_numpy()
y = candy['winpercent'].to_numpy()

In [9]:
from sklearn.model_selection import KFold

# Use KFold
kf = KFold(n_splits=5, shuffle=True, random_state=1111)

# Create splits
splits = kf.split(X)

'''
What dose splits contain?
- The splits variable contains the training and validation 
  indices for the five different splits of X.

'''
# Print the number of indices
for train_index, val_index in splits:
    print("Number of training indices: %s" % len(train_index))
    print("Number of validation indices: %s" % len(val_index))

Number of training indices: 68
Number of validation indices: 17
Number of training indices: 68
Number of validation indices: 17
Number of training indices: 68
Number of validation indices: 17
Number of training indices: 68
Number of validation indices: 17
Number of training indices: 68
Number of validation indices: 17


> **This dataset has 85 rows. You have created five splits - each containing 68 training and 17 validation indices. You can use these indices to complete 5-fold cross-validation.**

### Using `KFold` indices

You have already created `splits`, which contains __indices__ for the candy-data dataset to complete __5-fold cross-validation__. To get a better estimate for how well a colleague's random forest model will perform on a new data, you want to run this model on the five different __training__ and __validation__ indices you just created.

In this exercise, you will use these indices to check the accuracy of this model using the five different splits. A for loop has been provided to assist with this process.

In [10]:
# Create splits
splits = kf.split(X)

In [11]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

rfc = RandomForestRegressor(n_estimators=25, random_state=1111)

# Access the training and validation indices of splits
for train_index, val_index in splits:
    # Setup the training and validation data
    X_train, y_train = X[train_index], y[train_index]
    X_val, y_val = X[val_index], y[val_index]
    # Fit the random forest model
    rfc.fit(X_train, y_train)
    # Make predictions, and print the accuracy
    predictions = rfc.predict(X_val)
    print("Split accuracy: " + str(mean_squared_error(y_val, predictions)))

Split accuracy: 151.5028145199104
Split accuracy: 173.4624060357644
Split accuracy: 132.7340977072911
Split accuracy: 81.50364942339418
Split accuracy: 217.17904656079338


> __`KFold()` is a great method for accessing individual indices when completing cross-validation. One drawback is needing a for loop to work through the indices though.__

## sklearn's `cross_val_score()`

### scikit-learn's methods
You have decided to build a regression model to predict the number of new employees your company will successfully hire next month. You open up a new Python script to get started, but you quickly realize that sklearn has a lot of different modules. Let's make sure you understand the names of the modules, the methods, and which module contains which method.

Follow the instructions below to load in all of the necessary methods for completing cross-validation using sklearn. You will use modules:

- metrics
- model_selection
- ensemble

In [12]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, make_scorer

### Implement `cross_val_score()`

Your company has created several new candies to sell, but they are not sure if they should release all five of them. To predict the popularity of these new candies, you have been asked to build a regression model using the candy dataset. Remember that the response value is a head-to-head win-percentage against other candies.

Before you begin trying different regression models, you have decided to run cross-validation on a simple random forest model to get a baseline error to compare with any future results.

In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, make_scorer

rfc = RandomForestRegressor(n_estimators=25, random_state=1111)
mse = make_scorer(mean_squared_error)

# Setup cross_val_score
cv = cross_val_score(estimator=rfc,
                     X=X_train,
                     y=y_train,
                     cv=10,
                     scoring=mse)

# Print the mean error
print(cv.mean())

130.91371947185584


You now have a baseline score to build on. If you decide to build additional models or try new techniques, you should try to get an error lower than 155.56. Lower errors indicate that your popularity predictions are improving.

## Leave-one-out-cross-validation (LOOCV)
- **LOOCV**
In **leave-one-out-cross-validation**, we are going to implement `KFold` cross-validation, where `k` is equal to `n`, the number of observations in the data. 

<p align='center'>
    <a href='#'><img src='https://biol607.github.io/lectures/images/cv/loocv.png'></a>
</p>

- When to use LOOCV?
    - The amount of **training data is limited**
    - You want the absolute best error estimate for new data

- Be cautious when:
    - Computation resources are limited
    - You have a lot of data
    - You have a lot of parameters to test

- if you are planning on testing a lot of different parameter sets. The best way to judge if this method is even possible is to run __KFold__ cross-validation with a large __K__, maybe $25$ or $50$, and gauge how long it would take you to actually run Leave-one-out-cross-validation with the __n-observations__ in your data.

- We can find the __Number of observations__ by looking at the shape of the __X__ dataset. 



### Leave-one-out-cross-validation
Let's assume your favorite candy is not in the candy dataset, and that you are interested in the popularity of this candy. Using 5-fold cross-validation will train on only 80% of the data at a time. The candy dataset only has 85 rows though, and leaving out 20% of the data could hinder our model. However, using leave-one-out-cross-validation allows us to make the most out of our limited dataset and will give you the best estimate for your favorite candy's popularity!



In [18]:
y.shape[0]

85

In [16]:
from sklearn.metrics import mean_absolute_error, make_scorer

# Create scorer
mae_scorer = make_scorer(mean_absolute_error)

rfr = RandomForestRegressor(n_estimators=15, random_state=1111)

# Implement LOOCV
scores = cross_val_score(rfr, X=X, y=y, cv=y.shape[0], scoring=mae_scorer)

# Print the mean and standard deviation
print("The mean of the errors is: %s." % np.mean(scores))
print("The standard deviation of the errors is: %s." % np.std(scores))

The mean of the errors is: 9.464989603398694.
The standard deviation of the errors is: 7.265762094853885.


You have come along way with model validation techniques. The final chapter will wrap up model validation by discussing how to select the best model and give an introduction to parameter tuning.

## __Connect with Me__ 
--- 
[<img align="left" alt="codeSTACKr | Twitter" width="40px" src="https://cdn.jsdelivr.net/npm/simple-icons@v3/icons/twitter.svg" />][twitter] 
[<img align="left" alt="codeSTACKr | LinkedIn" width="40px" src="https://cdn.jsdelivr.net/npm/simple-icons@v3/icons/linkedin.svg" />][linkedin] 
[<img align="left" alt="codeSTACKr.com" width="40px" src="https://raw.githubusercontent.com/iconic/open-iconic/master/svg/globe.svg" />][StackExchange AI] 
[twitter]: https://twitter.com/F4izy 
--- 
[linkedin]: https://www.linkedin.com/in/mohd-faizy/ 
--- 
[StackExchange AI]: https://mohd-faizy.github.io
--- 
--- 
--- 
---